In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model

In [2]:
test_df = pd.read_csv("/content/Test_return_uncorr.csv")
train_df = pd.read_csv("/content/Train_return_uncorr.csv")
pred_df = pd.read_csv("/content/predicted_return_uncorr.csv")

In [3]:
train_df.head(10)

,Return_AES,Return_CAG,Return_CLX,Return_GOOG,Return_MSFT,Return_PM,Return_PNW,Return_SWKS,Return_UPS,Return_WMT
0,0.036916,0.001736,0.007429,0.015692,0.019307,0.008602,0.024317,-0.001953,0.006627,-0.001089
1,-0.009494,-0.005497,-0.002276,-0.002688,-0.005059,0.003819,-0.013566,0.025227,-0.003491,-0.012239
2,0.025559,0.017463,0.020987,0.004241,0.014071,0.007482,0.011564,0.024907,0.003403,0.021102
3,0.003115,0.002573,0.000357,-0.002756,-0.000583,0.005160,0.001390,-0.002842,-0.003691,0.000840
4,-0.014752,-0.011982,-0.007058,0.020349,0.001050,-0.029176,-0.014193,-0.002948,-0.006207,-0.003122
5,-0.014972,0.008951,-0.009897,-0.019978,-0.033566,-0.010706,-0.013302,-0.058052,-0.018134,-0.020828
6,-0.003200,-0.005436,-0.003635,-0.006496,-0.006030,-0.004694,-0.007772,-0.026473,-0.008414,0.007009
7,0.001605,0.057842,0.003466,-0.012282,-0.005824,0.005895,0.007832,0.062231,-0.000517,-0.006594
8,0.020833,0.003807,0.006726,0.006730,-0.000244,0.000912,0.011895,0.013154,0.007040,0.014504
9,0.008634,-0.010027,-0.003341,-0.007302,-0.007446,-0.019906,-0.000784,-0.018376,-0.003393,-0.003391


In [4]:
# Convert DataFrames to Tensors
train_data = tf.convert_to_tensor(train_df.values, dtype=tf.float32)
test_data  = tf.convert_to_tensor(test_df.values, dtype=tf.float32)
pred_data  = tf.convert_to_tensor(pred_df.values, dtype=tf.float32)

In [5]:
def preprocess_combined_returns_10(data):
    """
    Preprocess the combined returns data for 10 stocks arranged as:
      ["WMT", "MSFT", "UPS", "GOOG", "PM", "CLX", "CAG", "AES", "PNW", "SWKS"]

    Large cap stocks: first 5 columns
    Small cap stocks: last 5 columns

    Returns:
      Tensor of shape (batch, 2) where:
        - Column 0 is the aggregated large-cap return (mean across large cap stocks)
        - Column 1 is the aggregated small-cap return (mean across small cap stocks)
    """
    # Indices for each group
    large_cap_indices = list(range(0, 5))
    small_cap_indices = list(range(5, 10))

    # Extract and aggregate returns
    returns_large = tf.gather(data, indices=large_cap_indices, axis=1)
    returns_small = tf.gather(data, indices=small_cap_indices, axis=1)

    agg_large = tf.reduce_mean(returns_large, axis=1, keepdims=True)
    agg_small = tf.reduce_mean(returns_small, axis=1, keepdims=True)

    # Concatenate the aggregates into a (batch, 2) tensor
    return tf.concat([agg_small, agg_large], axis=1)

In [6]:
train_returns_combined = preprocess_combined_returns_10(train_data)
test_returns_combined  = preprocess_combined_returns_10(test_data)
pred_returns_combined  = preprocess_combined_returns_10(pred_data)

In [7]:
print(train_returns_combined[1])

tf.Tensor([-5.0088205e-05 -5.0028772e-03], shape=(2,), dtype=float32)


In [8]:
print("Train returns shape:", train_df.shape)
print("Test returns shape:", test_df.shape)
print("Pred returns shape:", pred_df.shape)

Train returns shape: (2135, 10)
Test returns shape: (330, 10)
Pred returns shape: (330, 10)


In [9]:
print("Train returns shape:", train_returns_combined.shape)
print("Test returns shape:", test_returns_combined.shape)
print("Pred returns shape:", pred_returns_combined.shape)

Train returns shape: (2135, 2)
Test returns shape: (330, 2)
Pred returns shape: (330, 2)


In [10]:
class PortfolioOptimizer(Model):
    def __init__(self, embed_dim=16, num_heads=2, ff_dim=32, num_layers=2):
        super().__init__()

        self.embedding = layers.Dense(embed_dim, activation="relu")

        self.transformer_layers = []
        for _ in range(num_layers):
            mha = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
            ff = tf.keras.Sequential([
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ])
            norm1 = layers.LayerNormalization(epsilon=1e-6)
            norm2 = layers.LayerNormalization(epsilon=1e-6)
            self.transformer_layers.append((mha, ff, norm1, norm2))

        # Final output => 2 raw weights: [w_long, w_short]
        self.output_layer = layers.Dense(2, activation=None)

    def call(self, inputs):
        """
        inputs => shape (batch, 2) => [R_long, R_short]
        returns => shape (batch, 2) => [w_long, w_short]
        """
        # 1) Embedding => (batch, embed_dim)
        x = self.embedding(inputs)

        # 2) Add a "time" dimension => (batch, 1, embed_dim)
        x = tf.expand_dims(x, axis=1)

        # 3) Pass through transformer blocks
        for (mha, ff, norm1, norm2) in self.transformer_layers:
            attn_output = mha(x, x)  # self-attention
            x = norm1(x + attn_output)
            ff_out = ff(x)
            x = norm2(x + ff_out)

        # 4) Squeeze time dim => (batch, embed_dim)
        x = tf.squeeze(x, axis=1)

        # 5) Final => (batch, 2)
        w_raw = self.output_layer(x)
        return w_raw

In [11]:
def custom_portfolio_loss(returns, weights,
                          delta_sum=10.0,    # penalty coefficient for w_long + w_short ≠ 1
                          beta=10.0,         # penalty coefficient for credit spread violation
                          gamma_long=10.0,   # penalty for w_long out of range
                          gamma_short=10.0   # penalty for w_short out of range
                         ):
    """
    Parameters:
      returns: Tensor of shape (batch, 2) = [R_long, R_short]
      weights: Tensor of shape (batch, 2) = [w_long, w_short]

    Constraints:
      1) w_long + w_short = 1
      2) R_long - R_short > 0 (i.e. credit spread must be positive)
      3) w_long ∈ [1.1, 1.3] and w_short ∈ [-0.3, -0.1]

    The loss is defined as:
       loss = -Sharpe + (penalty for weight sum) + (penalty for credit spread)
              + (penalty for w_long range) + (penalty for w_short range)
    """
    w_long  = weights[:, 0]
    w_short = weights[:, 1]

    # Calculate portfolio returns and Sharpe ratio:
    port_ret = w_long * returns[:, 0] + w_short * returns[:, 1]
    mean_ret = tf.reduce_mean(port_ret)
    std_ret  = tf.math.reduce_std(port_ret) + 1e-6  # avoid div-by-zero
    sharpe   = mean_ret / std_ret
    loss_sharpe = -sharpe  # we want to maximize Sharpe

    # 1) Penalty for weight sum: (w_long + w_short) should equal 1.
    penalty_sum = tf.reduce_mean(tf.square((w_long + w_short) - 1.0))
    loss_sum = delta_sum * penalty_sum

    # 2) Credit spread penalty: We want R_long - R_short > 0.
    #    Penalize when R_short - R_long > 0.
    penalty_spread = tf.reduce_mean(tf.nn.relu(returns[:, 1] - returns[:, 0]))
    loss_spread = beta * penalty_spread

    # 3) Weight range penalties:
    #    For w_long: desired range [1.0, 1.3]
    penalty_w_long_lower = tf.nn.relu(1.0 - w_long)  # >0 if w_long is below 1.1
    penalty_w_long_upper = tf.nn.relu(w_long - 1.3)    # >0 if w_long is above 1.3
    loss_w_long = gamma_long * tf.reduce_mean(tf.square(penalty_w_long_lower + penalty_w_long_upper))

    #    For w_short: desired range [-0.3, -0.1]
    penalty_w_short_lower = tf.nn.relu(-0.3 - w_short)  # >0 if w_short is less than -0.3
    penalty_w_short_upper = tf.nn.relu(w_short + 0.1)     # >0 if w_short is greater than -0.1
    loss_w_short = gamma_short * tf.reduce_mean(tf.square(penalty_w_short_lower + penalty_w_short_upper))

    # Total loss:
    total_loss = loss_sharpe + loss_sum + loss_spread + loss_w_long + loss_w_short
    return total_loss

In [12]:
# Example hyperparams
batch_size = 64
epochs = 10
lr = 1e-3

In [13]:
model = PortfolioOptimizer(embed_dim=16, num_heads=2, ff_dim=32, num_layers=2)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

In [14]:
num_samples = train_returns_combined.shape[0]
steps_per_epoch = num_samples // batch_size

In [15]:
for epoch in range(epochs):
    # Shuffle
    idx = tf.random.shuffle(tf.range(num_samples))
    epoch_loss = 0.0

    for step in range(steps_per_epoch):
        # Mini-batch
        batch_idx = idx[step * batch_size : (step+1) * batch_size]
        batch_returns = tf.gather(train_returns_combined, batch_idx)

        with tf.GradientTape() as tape:
            w_pred = model(batch_returns)  # (batch, 2): [w_long, w_short]

            loss_value = custom_portfolio_loss(
                returns=batch_returns,
                weights=w_pred,
                delta_sum=10.0,    # penalty coefficient for w_long + w_short ≠ 1
                beta=10.0,         # penalty coefficient for credit spread violation
                gamma_long=10.0,   # penalty for w_long out of range
                gamma_short=10.0
            )

        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        epoch_loss += loss_value.numpy()

    avg_loss = epoch_loss / steps_per_epoch
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss = {avg_loss.item():.4f}")

Epoch 1/10 - Avg Loss = 9.1797
Epoch 2/10 - Avg Loss = 0.1198
Epoch 3/10 - Avg Loss = 0.0473
Epoch 4/10 - Avg Loss = 0.0269
Epoch 5/10 - Avg Loss = 0.0155
Epoch 6/10 - Avg Loss = 0.0062
Epoch 7/10 - Avg Loss = -0.0115
Epoch 8/10 - Avg Loss = -0.0043
Epoch 9/10 - Avg Loss = -0.0055
Epoch 10/10 - Avg Loss = -0.0091


In [16]:
pred_weights = model(pred_returns_combined)

In [17]:
print("Test Returns (R_long, R_short):\n", pred_returns_combined.numpy())
print("Predicted Weights (w_long, w_short):\n", pred_weights.numpy())

Test Returns (R_long, R_short):
 [[ 1.5276212e-03  2.0699643e-03]
 [ 1.0488597e-03  1.5047553e-03]
 [ 6.9623918e-04  8.8765303e-04]
 [ 1.7111946e-03  2.0405692e-03]
 [ 1.0048642e-03  1.7781451e-03]
 [ 1.7691139e-03  2.5020479e-03]
 [ 1.6866217e-03  2.5004093e-03]
 [ 1.5448261e-03  2.3976420e-03]
 [ 7.5800368e-04  2.3867493e-03]
 [ 1.1915233e-03  1.9363839e-03]
 [ 1.3500158e-03  1.8648937e-03]
 [ 2.2805904e-03  2.1796850e-03]
 [ 2.3185047e-03  2.0734540e-03]
 [ 2.1531873e-03  1.8666203e-03]
 [ 2.3259684e-03  1.9849492e-03]
 [ 1.4996158e-03  1.6763326e-03]
 [ 2.0307982e-03  1.8896016e-03]
 [ 1.3996346e-03  1.5538613e-03]
 [ 1.6398616e-03  1.9862824e-03]
 [ 2.6592247e-03  2.2419770e-03]
 [ 2.8372451e-03  2.3295181e-03]
 [ 2.7878887e-03  2.3700586e-03]
 [ 2.8572450e-03  2.3667838e-03]
 [ 2.3590981e-03  2.2101956e-03]
 [ 2.1731572e-03  2.2633143e-03]
 [ 1.9470606e-03  2.2898368e-03]
 [ 1.8917400e-03  2.0920476e-03]
 [ 2.1720594e-03  2.5330740e-03]
 [ 2.0873833e-03  2.5258113e-03]
 [ 1.80853

In [18]:
pred_returns_combined

<tf.Tensor: shape=(330, 2), dtype=float32, numpy=
array([[ 1.5276212e-03,  2.0699643e-03],
       [ 1.0488597e-03,  1.5047553e-03],
       [ 6.9623918e-04,  8.8765303e-04],
       [ 1.7111946e-03,  2.0405692e-03],
       [ 1.0048642e-03,  1.7781451e-03],
       [ 1.7691139e-03,  2.5020479e-03],
       [ 1.6866217e-03,  2.5004093e-03],
       [ 1.5448261e-03,  2.3976420e-03],
       [ 7.5800368e-04,  2.3867493e-03],
       [ 1.1915233e-03,  1.9363839e-03],
       [ 1.3500158e-03,  1.8648937e-03],
       [ 2.2805904e-03,  2.1796850e-03],
       [ 2.3185047e-03,  2.0734540e-03],
       [ 2.1531873e-03,  1.8666203e-03],
       [ 2.3259684e-03,  1.9849492e-03],
       [ 1.4996158e-03,  1.6763326e-03],
       [ 2.0307982e-03,  1.8896016e-03],
       [ 1.3996346e-03,  1.5538613e-03],
       [ 1.6398616e-03,  1.9862824e-03],
       [ 2.6592247e-03,  2.2419770e-03],
       [ 2.8372451e-03,  2.3295181e-03],
       [ 2.7878887e-03,  2.3700586e-03],
       [ 2.8572450e-03,  2.3667838e-03],
       

In [19]:
test_returns=test_returns_combined.numpy()

In [20]:
test_returns[1]

array([0.00999753, 0.0137669 ], dtype=float32)

In [21]:
portfolio_daily_returns = test_returns[:, 0] * pred_weights[:, 0] + test_returns[:, 1] * pred_weights[:, 1]

In [22]:
portfolio_daily_returns

<tf.Tensor: shape=(330,), dtype=float32, numpy=
array([-1.49608944e-02,  9.44106933e-03, -1.30478386e-02, -1.18882712e-02,
       -1.58977173e-02,  1.03178369e-02,  4.02644370e-03,  4.36053518e-03,
        1.02880634e-02, -2.37059640e-03, -8.30224529e-03, -3.45684076e-03,
       -1.31671308e-02, -6.88863359e-03,  5.94469486e-03, -4.07109177e-03,
        2.67035663e-02,  1.48950806e-02, -2.29816344e-02,  3.67202191e-03,
        3.37121123e-03,  2.73295533e-04,  8.31372570e-03,  5.73188672e-03,
       -2.38203676e-03,  5.61823044e-03,  9.96596413e-04,  2.56668846e-03,
        6.87953830e-03,  1.15784165e-03, -1.94989634e-03,  2.46342155e-03,
        6.46262662e-04, -3.36056692e-04,  1.30678657e-02, -6.15804596e-03,
        2.13393793e-02,  9.21324082e-03, -3.01419036e-03, -4.65827715e-03,
        2.29685893e-03, -2.48866398e-02,  8.08147993e-03,  6.00318750e-03,
        3.89632210e-03,  4.15864110e-04,  3.00548668e-03, -3.09834210e-03,
        4.47685504e-03, -5.76721458e-03, -4.69879713

In [23]:
portfolio_returns = portfolio_daily_returns.numpy()

In [24]:
portfolio_returns

array([-1.49608944e-02,  9.44106933e-03, -1.30478386e-02, -1.18882712e-02,
       -1.58977173e-02,  1.03178369e-02,  4.02644370e-03,  4.36053518e-03,
        1.02880634e-02, -2.37059640e-03, -8.30224529e-03, -3.45684076e-03,
       -1.31671308e-02, -6.88863359e-03,  5.94469486e-03, -4.07109177e-03,
        2.67035663e-02,  1.48950806e-02, -2.29816344e-02,  3.67202191e-03,
        3.37121123e-03,  2.73295533e-04,  8.31372570e-03,  5.73188672e-03,
       -2.38203676e-03,  5.61823044e-03,  9.96596413e-04,  2.56668846e-03,
        6.87953830e-03,  1.15784165e-03, -1.94989634e-03,  2.46342155e-03,
        6.46262662e-04, -3.36056692e-04,  1.30678657e-02, -6.15804596e-03,
        2.13393793e-02,  9.21324082e-03, -3.01419036e-03, -4.65827715e-03,
        2.29685893e-03, -2.48866398e-02,  8.08147993e-03,  6.00318750e-03,
        3.89632210e-03,  4.15864110e-04,  3.00548668e-03, -3.09834210e-03,
        4.47685504e-03, -5.76721458e-03, -4.69879713e-03,  1.37128215e-03,
        1.07371313e-02, -

In [33]:
risk_free_rate = 0.05  # 5%

In [34]:
initial_value = 1.0
face = initial_value
T = len(portfolio_returns)  # total number of days

for ret in portfolio_returns:
    face *= (1 + ret)

In [35]:
annualized_return = (face ** (252 / T) - 1)- risk_free_rate

In [36]:
import statistics
import math

if T > 1:
    daily_std = statistics.stdev(portfolio_returns)
else:
    daily_std = 0.0

annualized_std = daily_std * math.sqrt(252)


In [37]:
if annualized_std != 0:
    annualized_sharpe = (annualized_return ) / annualized_std
else:
    annualized_sharpe = float('inf')  # or handle zero volatility appropriately


In [38]:
print("Annualized Return: {:.4%}".format(annualized_return))
print("Annualized Std Dev: {:.4%}".format(annualized_std))
print("Annualized Sharpe Ratio: {:.4f}".format(annualized_sharpe))

Annualized Return: 11.2641%
Annualized Std Dev: 15.1423%
Annualized Sharpe Ratio: 0.7439
